## 导入库

In [1]:
# -*- coding: utf-8 -*-
from __future__ import annotations

# ── 标准库 ──────────────────────────────────────────────────────────────────
import os
import time
from pathlib import Path
from collections import defaultdict
from datetime import datetime
import re

# ── 第三方 ──────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd
import polars as pl

import gc

import torch
import torch.backends.cudnn as cudnn
import lightning as L
import lightning.pytorch as lp
from torch.utils.data import DataLoader, RandomSampler

from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import DeviceStatsMonitor
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, Baseline
from pytorch_forecasting.metrics import MAE, RMSE, QuantileLoss
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_forecasting.data import TorchNormalizer, GroupNormalizer


# 你的工程工具
from pipeline.io import cfg, P, fs, storage_options, ensure_dir_local, ensure_dir_az
from pipeline.stream_input_local import ShardedBatchStream  
from pipeline.wr2 import WR2

# ---- 性能/兼容开关（仅一次）----
os.environ.setdefault("POLARS_MAX_THREADS", str(max(1, os.cpu_count() // 2)))
pl.enable_string_cache()
cudnn.benchmark = True
torch.set_float32_matmul_precision("high")

import time as _t


import warnings
warnings.filterwarnings("ignore")  # avoid printing out absolute paths

def _now() -> str:
    return _t.strftime("%Y-%m-%d %H:%M:%S")
print(f"[{_now()}] imports ok")

/home/admin_ml/Jackson/projects/js/JS/.venv/lib/python3.10/site-packages/pytorch_forecasting/models/base/_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


[2025-10-27 12:09:02] imports ok


## 定义工具函数

In [2]:
# ───────────────────────────────────────────────────────────────────────────
# 滑动窗划分
def make_sliding_cv_by_days(all_days: np.ndarray, *, n_splits: int, gap_days: int, train_to_val: int):
    all_days = np.asarray(all_days).ravel()
    K, R, G = n_splits, train_to_val, gap_days
    usable = len(all_days) - G
    if usable <= 0 or K <= 0 or R <= 0:
        return []
    V_base, rem = divmod(usable, R + K)
    if V_base <= 0:
        return []
    T = R * V_base
    v_lens = [V_base + 1 if i < rem else V_base for i in range(K)]
    folds, v_lo = [], T + G
    for V_i in v_lens:
        v_hi, tr_hi, tr_lo = v_lo + V_i, v_lo - G, v_lo - G - T
        if tr_lo < 0 or v_hi > len(all_days):
            break
        folds.append((all_days[tr_lo:tr_hi], all_days[v_lo:v_hi]))
        v_lo = v_hi
    return folds


## 初始化参数

读入选出的特征列

In [3]:
# 读入筛选的所有特征列

path = Path("/mnt/data/js/exp/v1/models/tune/selected_covariant_features.txt")
filted_cols = path.read_text(encoding="utf-8").splitlines()

selected_features = pd.read_csv("/mnt/data/js/exp/v1/models/tune/feature_importance__fixed__fixed__mm_full_train__features__fs__1400-1698__cv2-g7-r4__seed42__top1000__1760906660__range830-1698__range830-1698__cv2-g7-r4__1760912739.csv")

df_cov_cols = selected_features[selected_features['feature'].isin(filted_cols)].copy()

# 我们这里重新归一化一下
df_cov_cols["mean_gain"] = (df_cov_cols['mean_gain'] / df_cov_cols['mean_gain'].sum()).astype(np.float32)
df_e_features = df_cov_cols.reset_index(drop=True)

In [4]:
# ========== 1) 初始化配置 ==========

# 所有列
G_SYM, G_DATE, G_TIME = cfg["keys"]          # e.g. ("symbol_id","date_id","time_id")
TARGET_COL = cfg["target"]                   # e.g. "responder_6"
WEIGHT_COL = cfg["weight"]                   # 允许为 None

TIME_FEATURES = ["time_bucket", "time_pos", "time_sin", "time_cos"]
COV_FEATURES = df_e_features['feature'].tolist() # 含有常数列


STATIC_FEATURES = [c for c in COV_FEATURES if c in ["feature_09", "feature_10", "feature_11"]]

CS_RANK_FEATURES = [c for c in COV_FEATURES if c.endswith("__csrank")]

CS_R_Z_FEATURES = [c for c in COV_FEATURES if c.endswith("__cs_z") or c.endswith("__rz")]

do_z_co_cols = [c for c in COV_FEATURES if c not in STATIC_FEATURES + CS_RANK_FEATURES + CS_R_Z_FEATURES]


# 训练 & CV 超参
N_SPLITS     = 2
GAP_DAYS     = 7
TRAIN_TO_VAL = 5
ENC_LEN      = 128
DEC_LEN      = 1
PRED_LEN     = DEC_LEN
BATCH_SIZE   = 64 # 有效batch_size = 128
ACCUM        = 2
LR           = 5e-4
HIDDEN       = 64
HEADS        = 2
DROPOUT      = 0.2
MAX_EPOCHS   = 40
GCV = 0.1
precision="16-mixed"

# 数据路径
PANEL_DIR_AZ   = P("az", cfg["paths"].get("panel_shards", "panel_shards"))

TFT_LOCAL_ROOT = P("local", "tft"); ensure_dir_local(TFT_LOCAL_ROOT)

LOCAL_CLEAN_DIR = f"{TFT_LOCAL_ROOT}/clean"; ensure_dir_local(LOCAL_CLEAN_DIR)
CKPTS_DIR = Path(TFT_LOCAL_ROOT) / "ckpts"; ensure_dir_local(CKPTS_DIR.as_posix())
LOGS_DIR  = Path(TFT_LOCAL_ROOT) / "logs";  ensure_dir_local(LOGS_DIR.as_posix())


start_date, end_date = (1000, 1600)  # 
s_start_data, s_end_date = 1400, 1600

print("[config] ready")

[config] ready


## 数据导入

In [ ]:
data_paths = fs.glob("az://jackson/js_exp/exp/v1/panel_shards/*.parquet")
data_paths =[f"az://{p}" for p in data_paths]

lf_data = (
    pl.scan_parquet(data_paths, storage_options=storage_options)
    .select([*cfg['keys'], WEIGHT_COL, TARGET_COL, *TIME_FEATURES, *COV_FEATURES])
    .filter(pl.col(G_DATE).is_between(start_date, end_date, closed="both"))
)
lf_data = lf_data.sort([G_SYM, G_DATE, G_TIME])


## 数据处理

### 添加全局时间序列号

In [ ]:
grid_df = (
    lf_data.select([G_DATE, G_TIME]).unique()
        .sort([G_DATE, G_TIME])
        .with_row_index("time_idx")
        .with_columns(pl.col("time_idx").cast(pl.Int64))
        .collect(streaming=True)
)

In [ ]:
container_prefix = "az://jackson/js_exp/exp/v1/tft/panel_clean_shards"; ensure_dir_az(container_prefix)
chunk_size = 30
for lo in range(start_date, end_date + 1, chunk_size):
    hi = min(lo + chunk_size - 1, end_date)
    print(f"processing date range: {lo} ~ {hi}")
    
    lf_chunk = lf_data.filter(pl.col(G_DATE).is_between(lo, hi, closed="both"))
    
    lf_grid_chunk = (
        grid_df.lazy().filter(pl.col(G_DATE).is_between(lo, hi, closed="both"))
    )
    
    lf_joined = (
        lf_chunk.join(lf_grid_chunk, on=[G_DATE, G_TIME], how="left").sort([G_SYM, "time_idx"])
    )
    
    out_path = f"{container_prefix}/panel_clean_{lo:04d}_{hi:04d}.parquet"
    print(f"writing to: {out_path}")
    
    lf_joined.sink_parquet(
        out_path,
        storage_options=storage_options,
        compression="zstd",
    )
print(f"[{_now()}] all done")

## 导入新数据

In [5]:
# 重新读入数据
container_prefix = "az://jackson/js_exp/exp/v1/tft/panel_clean_shards"; ensure_dir_az(container_prefix)
data_paths = fs.glob(f"{container_prefix}/*.parquet")
data_paths = [f"az://{p}" for p in data_paths] 
lf_with_idx = pl.scan_parquet(data_paths, storage_options=storage_options)
lf_with_idx = lf_with_idx.filter(pl.col(G_DATE).is_between(s_start_data, s_end_date)).sort([G_SYM, G_DATE, G_TIME])

## CV 划分

In [6]:
# ==========  CV 划分 ==========
all_days = (
    lf_with_idx.select(pl.col(G_DATE)).unique().sort([G_DATE])
    .collect(streaming=True)[G_DATE].to_numpy()
)
folds_by_day = make_sliding_cv_by_days(all_days, n_splits=N_SPLITS, gap_days=GAP_DAYS, train_to_val=TRAIN_TO_VAL)

print(f"[cv] total {len(folds_by_day)} folds")

assert len(folds_by_day) > 0, "no CV folds constructed"
print(folds_by_day)

[cv] total 2 folds
[(array([1400, 1401, 1402, 1403, 1404, 1405, 1406, 1407, 1408, 1409, 1410,
       1411, 1412, 1413, 1414, 1415, 1416, 1417, 1418, 1419, 1420, 1421,
       1422, 1423, 1424, 1425, 1426, 1427, 1428, 1429, 1430, 1431, 1432,
       1433, 1434, 1435, 1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443,
       1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453, 1454,
       1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464, 1465,
       1466, 1467, 1468, 1469, 1470, 1471, 1472, 1473, 1474, 1475, 1476,
       1477, 1478, 1479, 1480, 1481, 1482, 1483, 1484, 1485, 1486, 1487,
       1488, 1489, 1490, 1491, 1492, 1493, 1494, 1495, 1496, 1497, 1498,
       1499, 1500, 1501, 1502, 1503, 1504, 1505, 1506, 1507, 1508, 1509,
       1510, 1511, 1512, 1513, 1514, 1515, 1516, 1517, 1518, 1519, 1520,
       1521, 1522, 1523, 1524, 1525, 1526, 1527, 1528, 1529, 1530, 1531,
       1532, 1533, 1534], dtype=int32), array([1542, 1543, 1544, 1545, 1546, 1547, 1548, 1549, 1550, 15

下面我们以一折为例，验证成功后，再使用多折

In [7]:
fold_id = 0
# 取第一个 fold 的训练集最后一天，作为本折统计 z-score 的上界
train_lo, train_hi = folds_by_day[fold_id][0][0], folds_by_day[fold_id][0][-1]
val_lo, val_hi = folds_by_day[fold_id][1][0], folds_by_day[fold_id][1][-1]

print(f"统计标准化使用训练集的日期范围 = {train_lo} ~ {train_hi}")

统计标准化使用训练集的日期范围 = 1400 ~ 1534


## 数据标准化处理

In [ ]:
# 先归一化 time_pos

lf_full = lf_with_idx.filter(pl.col(G_DATE).is_between(train_lo, val_hi, closed="both"))

lf_full = lf_full.with_columns(
    (pl.col("time_pos") / 968).alias("time_pos")
)

In [ ]:
lf_tr = lf_with_idx.filter(pl.col(G_DATE).is_between(train_lo, train_hi, closed="both"))

# ========= 归一化静态特征 ==========
static_glb_minmax = (
    lf_tr.select(
        *[pl.col(c).min().alias(f"{c}_min") for c in STATIC_FEATURES],
        *[pl.col(c).max().alias(f"{c}_max") for c in STATIC_FEATURES],
    ).collect().to_dicts()[0]   # ← 用 to_dicts()[0]
)

eps = 1e-8

lf_full = lf_full.with_columns([
    (
        ((pl.col(c) - pl.lit(static_glb_minmax[f"{c}_min"])) /
         (pl.lit(static_glb_minmax[f"{c}_max"] - static_glb_minmax[f"{c}_min"]) + eps))
        .clip(0.0, 1.0)
    ).cast(pl.Float32).alias(c)   # ← cast/alias 作用于“整个结果”
    for c in STATIC_FEATURES
])

del static_glb_minmax; gc.collect()


In [ ]:
# ========== Z-score ==========
# 开始计算 stats

In [ ]:
grp_stats = (
    lf_tr
    .group_by(G_SYM)
    .agg([pl.col(c).mean().alias(f"mu_grp_{c}") for c in do_z_co_cols] +
        [pl.col(c).std(ddof=1).alias(f"std_grp_{c}") for c in do_z_co_cols])
).collect(streaming=True)

In [ ]:
glb_stats = (
    lf_tr
    .select([pl.col(c).mean().alias(f"mu_glb_{c}") for c in do_z_co_cols] +
            [pl.col(c).std(ddof=1).alias(f"std_glb_{c}") for c in do_z_co_cols])
).collect(streaming=True)

glb_row = glb_stats.to_dicts()[0]

In [ ]:
import math

# 逐日处理本折所有数据
fold_root = f"az://jackson/js_exp/exp/v1/tft/fold_{fold_id}"; ensure_dir_az(fold_root)
z_prefix = f"{fold_root}/start{s_start_data}_train{s_end_date}_z_shards"; ensure_dir_az(z_prefix)

min_std = 1e-5
eps = 1e-6
# 预处理全局均值/方差的兜底，避免在行级判断
glb_mu = {c: glb_row[f"mu_glb_{c}"] for c in do_z_co_cols}
glb_std = {}
for c in do_z_co_cols:
    s = glb_row[f"std_glb_{c}"]
    if s is None or s <= 0:
        s = min_std
    glb_std[c] = s
z_done_cols = [f"z_{c}" for c in do_z_co_cols]
for d in range(train_lo, val_hi + 1):
    lf_day = lf_full.filter(pl.col(G_DATE) == d)

    lf_day_z = lf_day.join(grp_stats.lazy(), on=G_SYM, how="left")

    # 1) 先把所有“常量”变成具名列，避免 anonymous literal
    const_exprs = []
    for c in do_z_co_cols:
        const_exprs += [
            pl.lit(glb_mu[c]).alias(f"__mu_glb_{c}"),
            pl.lit(glb_std[c] if glb_std[c] > 0 else min_std).alias(f"__std_glb_{c}"),
        ]
    lf_day_z = lf_day_z.with_columns(const_exprs)

    # 2) 计算 z，并用 clip 裁剪
    exprs = []
    for c in do_z_co_cols:
        mu_grp  = pl.col(f"mu_grp_{c}")
        std_grp = pl.col(f"std_grp_{c}")
        mu_use = pl.coalesce([mu_grp, pl.col(f"__mu_glb_{c}")]).cast(pl.Float32)

        std_tmp = pl.when(std_grp.is_null() | (std_grp <= 0))\
                    .then(pl.col(f"__std_glb_{c}"))\
                    .otherwise(std_grp)
        std_use = pl.when(std_tmp < min_std).then(min_std).otherwise(std_tmp).cast(pl.Float32)

        z = ((pl.col(c).cast(pl.Float32) - mu_use) / (std_use + eps)).clip(-8.0, 8.0).alias(f"z_{c}")
        exprs.append(z)

    lf_day_z = lf_day_z.with_columns(exprs)
    
    # 标准化后，用0填补缺失值
    lf_day_z = lf_day_z.with_columns([
        pl.col(c).fill_null(pl.lit(0.0)).cast(pl.Float32) for c in [*z_done_cols, *CS_RANK_FEATURES, *CS_R_Z_FEATURES]
    ])
    
    lf_day_z = lf_day_z.sort([G_SYM, "time_idx"]).with_columns([
        pl.col(c).forward_fill().over(G_SYM) for c in STATIC_FEATURES
    ]).with_columns([
        pl.col(c).fill_null(pl.lit(0.0)).cast(pl.Float32) for c in STATIC_FEATURES
    ])
    
    keep = [
        "time_idx", G_SYM, G_DATE, G_TIME, WEIGHT_COL, TARGET_COL, *TIME_FEATURES, *STATIC_FEATURES, *CS_RANK_FEATURES, *CS_R_Z_FEATURES, *z_done_cols
    ]
    
    lf_out = lf_day_z.select(keep).sort([G_SYM, "time_idx"])

    out_path = f"{z_prefix}/z_{d:04d}.parquet"
    lf_out.collect(streaming=True).write_parquet(
        out_path, storage_options=storage_options, compression="zstd"
    )
    print(f"wrote z-scored data for day {d} to {out_path}")


### 检查缺失值

In [ ]:
# 检查缺失值
z_paths = fs.glob(f"{z_prefix}/*.parquet")
z_paths = [f"az://{p}" for p in z_paths]
lx = pl.scan_parquet(z_paths, storage_options=storage_options).sort([G_SYM, "time_idx"])


df_null = lx.select(
    [pl.col(c).null_count().alias(c) for c in CS_RANK_FEATURES]
).collect()

df_null = df_null.to_pandas()
df_null.rename(columns={0: "null_count"}, inplace=True)

df_null = df_null.T
df_null.rename(columns={0: "null_count"}, inplace=True)
df_null.sort_values(by="null_count", ascending=False)

In [ ]:
# 检查标准差
stds = lx.select(
    [pl.col(c).std().alias(c) for c in z_done_cols]
).collect(streaming=True)

pd_stds = stds.to_pandas().T

pd_stds.sort_values(by=0, ascending=True).head()

## PCA 降维

In [ ]:
z_done_cols = [f"z_{c}" for c in do_z_co_cols]

keep = [
    "time_idx", G_SYM, G_DATE, G_TIME, WEIGHT_COL, TARGET_COL, *TIME_FEATURES, *STATIC_FEATURES, *CS_RANK_FEATURES, *CS_R_Z_FEATURES, *z_done_cols
]

In [ ]:
# 确定各个特征列处理方案：
# already_z_cols = csrank_cols + other_already_z_cols

# 挑选出不需要PCA的列：看z_done_cols 和other_already_z_cols中哪些 由COV_FEATURES[:200] 衍生的
if False:
        topk_cols = COV_FEATURES[:200]

        csrank_cols = [c for c in already_z_cols if "__csrank" in c]
        other_already_z_cols = [c for c in already_z_cols if c not in csrank_cols]
        no_pca_cols_from_other_already = [c for c in other_already_z_cols if c in topk_cols]
        no_pca_cols_from_z_done = [f"z_{c}" for c in do_z_cols if c in topk_cols]
        no_pca_cols = ["time_idx", G_SYM, G_DATE, G_TIME, WEIGHT_COL, TARGET_COL,
                "time_sin", "time_cos", "time_bucket", "z_time_pos"] + no_z_grp_static_cols + csrank_cols + no_pca_cols_from_other_already + no_pca_cols_from_z_done
        no_pca_cols = list(dict.fromkeys(no_pca_cols))  # 去重且保持顺序

pca_cols =  TIME_FEATURES + STATIC_FEATURES + CS_RANK_FEATURES + CS_R_Z_FEATURES + z_done_cols
no_pca_cols = [c for c in ["time_idx", G_SYM, G_DATE, G_TIME, WEIGHT_COL, TARGET_COL]]

In [ ]:
# 训练集上计算 PCA

from sklearn.decomposition import IncrementalPCA
import joblib
# 逐日处理本折所有数据
fold_root = f"az://jackson/js_exp/exp/v1/tft/fold_{fold_id}"
z_prefix = f"{fold_root}/start{s_start_data}_train{s_end_date}_z_shards"

max_components = min(200, len(pca_cols))

ipca = IncrementalPCA(n_components=max_components)

for d in range(train_lo, train_hi + 1): 
    path = f"{z_prefix}/z_{d:04d}.parquet"
    print(f"fitting IPCA on date: {d} from the path: {path}")
    
    df_day = pl.read_parquet(path, storage_options=storage_options).select(pca_cols)
    X = df_day.to_numpy()
    ipca.partial_fit(X)  # 增量拟合


In [ ]:
# 拟合完成后，看看累计方差占比，决定真正保留的维度k (95%)
cum = ipca.explained_variance_ratio_.cumsum()
tau =0.85
k = int(np.searchsorted(cum, tau)) + 1
print(f"PCA components k = {k} for tau = {tau}")


In [ ]:
k_final = 192
print(f"保留主成分维度： {k_final}")

# 保存模型
# —— 裁剪参数到前 k 维（不要重新 new 模型）——
ipca.components_              = ipca.components_[:k_final]
ipca.explained_variance_      = ipca.explained_variance_[:k_final]
ipca.explained_variance_ratio_= ipca.explained_variance_ratio_[:k_final]
ipca.singular_values_         = ipca.singular_values_[:k_final]
ipca.n_components_            = k_final


pca_path = f"{TFT_LOCAL_ROOT}/fold_{fold_id}_ipca_{train_lo}_{train_hi}_k{k_final}_{_now()}.joblib"
joblib.dump(
    {"ipca": ipca, "pca_cols": pca_cols, "cum_ratio": cum, "chosen_k": k_final}, 
    pca_path
)
print(f"IPCA 模型已保存至: {pca_path}")
print()

In [ ]:
# 本折全部数据进行PCA变换

pca_prefix = f"{fold_root}/start{s_start_data}_train{s_end_date}_pca_shards"; ensure_dir_az(pca_prefix)
keep_cols = no_pca_cols

pca_k = ipca.n_components_
pca_out_cols = [f"PC{i+1}" for i in range(pca_k)]

for d in range(train_lo, val_hi + 1):
    in_path = f"{z_prefix}/z_{d:04d}.parquet"
    out_path = f"{pca_prefix}/pca_shard_{d:04d}.parquet"
    
    df = pl.read_parquet(in_path, storage_options=storage_options)
    
    X = (
        df.select(pca_cols).to_numpy()
    )
    if X.size == 0:
        # 当天没有行，直接跳过
        continue
    if np.isnan(X).any():
        raise ValueError("PCA transform 输入里仍有 NaN，请检查上游补值/标准化。")

    Z = ipca.transform(X).astype(np.float32)
    
    # 组装输出
    df_pca = pl.DataFrame(Z, schema=pca_out_cols)
    
    lf_out = pl.concat([
        df.select(no_pca_cols),
        df_pca
    ],
    how="horizontal"
    ).sort([G_SYM, "time_idx"])
    
    lf_out.write_parquet(
        out_path,
        storage_options=storage_options,
        compression="zstd"
    )
    print(f"written PCA features to: {out_path}")
print(f"[{_now()}] all done")


In [ ]:
# 按10天合并
ten_days_prefix = f"{fold_root}/feat_pca_ten_days_shards"; ensure_dir_az(ten_days_prefix)

chunk_size = 10

for lo in range(train_lo, val_hi + 1, chunk_size):
    hi = min(lo + chunk_size - 1, folds_by_day[fold_id][1][-1])
    paths = [f"{pca_prefix}/pca_shard_{d:04d}.parquet" for d in range(lo, hi + 1)]
    
    print(f"merging date range: {lo} ~ {hi}, num files = {len(paths)}")
    
    lf = pl.scan_parquet(paths, storage_options=storage_options)
    
    df = lf.collect(streaming=True).sort([G_SYM, "time_idx"]).rechunk()
    
    out_path = f"{ten_days_prefix}/feat_pca_ten_days_chunk_{lo:04d}_{hi:04d}.parquet"
    print(f"writing to: {out_path}")
    df.write_parquet(
        out_path,
        storage_options=storage_options,
        compression="zstd",
    )
print(f"[{_now()}] all done")

## 训练模型

In [8]:
# 读取处理好的数据
fold_root = f"az://jackson/js_exp/exp/v1/tft/fold_{fold_id}"
ten_days_prefix = f"{fold_root}/feat_pca_ten_days_shards"

lf_pca = pl.scan_parquet(f"{ten_days_prefix}/*.parquet", storage_options=storage_options)
lf_pca = lf_pca.sort([G_SYM, "time_idx"])

### 选择性PCA时

In [ ]:
if False:
        
    lf_clean_train = lf_clean.filter(pl.col(G_DATE).is_between(train_lo, train_hi, closed="both"))

    sym_med = lf_clean_train.group_by(G_SYM).agg(
        [pl.col(c).median().alias(f"median_{c}") for c in no_z_grp_static_cols]
    ).collect()

    glb_med = lf_clean_train.select(
        [pl.col(c).median().alias(f"glb_median_{c}") for c in no_z_grp_static_cols]
    ).collect()


    lf_pure = (
        lf_clean.join(sym_med.lazy(), on=G_SYM, how="left")
        .join(glb_med.lazy(), how="cross")
        .with_columns(
            [
                pl.coalesce([
                    pl.col(f"median_{c}"),
                    pl.col(f"glb_median_{c}")
                ]).alias(c)
                for c in no_z_grp_static_cols
            ]
            
        )
        .drop([f"median_{c}" for c in no_z_grp_static_cols] + [f"glb_median_{c}" for c in no_z_grp_static_cols])
        .sort([G_SYM, "time_idx"])
    )


    # 对Feature_09, 10, 11 标准化
    EPS = 1e-6
    cols = no_z_grp_static_cols
    # 训练期统计 -> 字典
    stats = (
        lf_pure.filter(pl.col(G_DATE).is_between(train_lo, train_hi, closed="both"))
            .select([pl.mean(c).alias(f"mean_{c}") for c in cols] +
                    [pl.std(c).alias(f"std_{c}")  for c in cols])
            .collect(streaming=True)
    )
    mu  = {c: float(stats[0, f"mean_{c}"]) for c in cols}
    sd  = {c: max(float(stats[0, f"std_{c}"]) or 0.0, EPS) for c in cols}


    lf_final = lf_pure.with_columns([
        ((pl.col(c) - pl.lit(mu[c])) / pl.lit(sd[c])).alias(c)   # 原地覆盖
        for c in cols
    ])
    
    KNOWN_CATEGORIES = ["time_bucket"]

    STATIC_REALS = [c for c in no_z_grp_static_cols]


    KNOWN_REALS = [c for c in lf_final.collect_schema().names() if c not in ([G_SYM, G_DATE, G_TIME, "time_idx", "time_bucket", WEIGHT_COL, TARGET_COL] + STATIC_REALS)]


    TRAIN_COLS = KNOWN_REALS + STATIC_REALS + KNOWN_CATEGORIES + [G_SYM, G_DATE, G_TIME, "time_idx", WEIGHT_COL, TARGET_COL] 

## try

In [ ]:
# 取第一折先试探一下
best_ckpt_paths, fold_metrics = [], []
fold_id = 0
train_days, val_days = folds_by_day[0]

print(f"[fold {fold_id}] train {train_days[0]}..{train_days[-1]} ({len(train_days)} days), "
    f"val {val_days[0]}..{val_days[-1]} ({len(val_days)} days)")

# 明确日期：
train_start_date = int(train_days[0])
train_end_date   = int(train_days[-1])
val_start_date   = int(val_days[0])
val_end_date     = int(val_days[-1])      


# 提取数据
date_range = (train_start_date, val_end_date)
pdf_data = (
    pl.scan_parquet(clean_path_local)
    .filter(pl.col(G_DATE).is_between(train_start_date, val_end_date, closed="both"))
    .collect(streaming=True)
    .to_pandas()
    .sort_values([G_SYM, "time_idx"])
)
pdf_data[G_SYM] = pdf_data[G_SYM].astype("str")
if "time_bucket" in pdf_data.columns:
    pdf_data["time_bucket"] = pdf_data["time_bucket"].astype("str")

# 明确 indexes:
train_end_idx = pdf_data.loc[pdf_data[G_DATE] == train_end_date, "time_idx"].max()
val_start_idx = pdf_data.loc[pdf_data[G_DATE] == val_start_date, "time_idx"].min()
val_end_idx   = pdf_data.loc[pdf_data[G_DATE] == val_end_date, "time_idx"].max()
assert pd.notna(train_end_idx) and pd.notna(val_start_idx) and pd.notna(val_end_idx), "train/val idx not found"
train_end_idx, val_start_idx, val_end_idx = int(train_end_idx), int(val_start_idx), int(val_end_idx)
print(f"[fold {fold_id}] train idx up to {train_end_idx}, val idx {val_start_idx}..{val_end_idx}")  



In [ ]:
t_data = pdf_data[TRAIN_COLS]

identity_scalers = {name: None for name in UNSCALE_COLS}
base_ds = TimeSeriesDataSet(
    t_data,
    time_idx="time_idx",
    target=TARGET_COL,
    group_ids=[G_SYM],
    weight=WEIGHT_COL,
    max_encoder_length=ENC_LEN, 
    min_encoder_length=ENC_LEN,
    max_prediction_length=PRED_LEN, 
    min_prediction_length=PRED_LEN,
    
    static_categoricals=[G_SYM],
    time_varying_known_categoricals=KNOWN_CATEGORIES,
    time_varying_known_reals =KNOWN_REALS,

    categorical_encoders={G_SYM: NaNLabelEncoder(add_nan=True),
                          "time_bucket": NaNLabelEncoder(add_nan=True) if "time_bucket" in KNOWN_CATEGORIES else None,
                          },
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    allow_missing_timesteps=True,
    target_normalizer=GroupNormalizer(
        method="standard", groups=[G_SYM], center=True, scale_by_group=False),
    scalers=identity_scalers,
)

In [ ]:
# 划分训练集，验证集
train_ds = base_ds.filter(
    lambda idx: (
        idx.time_idx_last <= train_end_idx
    ),
    copy=True
)

val_ds = base_ds.filter(
    lambda idx: (
        (idx.time_idx_first_prediction == val_start_idx + ENC_LEN) &
        
        (idx.time_idx_last <= val_end_idx)
    ),
    copy=True
)

In [ ]:
# 数据集加载

train_loader = train_ds.to_dataloader(
    train=True, 
    batch_size=BATCH_SIZE, 
    num_workers=14,
    pin_memory=True,
    persistent_workers=False,
    prefetch_factor=8,
)

n_train_batches = len(train_loader)
print(f"[debug] train_loader batches = {n_train_batches}")
assert n_train_batches > 0, "Empty train dataloader. Check min_prediction_idx/ENC_LEN/date windows."

val_loader = val_ds.to_dataloader(
    train=False,
    batch_size=BATCH_SIZE,
    num_workers=14,
    pin_memory=True,
    persistent_workers=True,
    prefetch_factor=8,
)

n_val_batches = len(val_loader)
print(f"[debug] val_loader batches = {n_val_batches}")
assert n_val_batches > 0, "Empty val dataloader. Check min_prediction_idx/ENC_LEN/date windows."

In [ ]:
lp.seed_everything(42)
trainer = lp.Trainer(
    accelerator="gpu",
    # clipping gradients is a hyperparameter and important to prevent divergance
    # of the gradient for recurrent neural networks
    gradient_clip_val=0.1,
)

tft = TemporalFusionTransformer.from_dataset(
    train_ds,
    # not meaningful for finding the learning rate but otherwise very important
    learning_rate=LR,
    hidden_size=HIDDEN,  # most important hyperparameter apart from learning rate
    # number of attention heads. Set to up to 4 for large datasets
    attention_head_size=HEADS,
    dropout=DROPOUT,  # between 0.1 and 0.3 are good values
    hidden_continuous_size=HIDDEN // 2,  # set to <= hidden_size
    loss=RMSE(),
    optimizer=torch.optim.Adam,
    # reduce learning rate if no improvement in validation loss after x epochs
    # reduce_on_plateau_patience=1000,
)
print(f"Number of parameters in network: {tft.size() / 1e3:.1f}k")
    
# find optimal learning rate
from lightning.pytorch.tuner import Tuner

res = Tuner(trainer).lr_find(
    tft,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    max_lr=10.0,
    min_lr=1e-6,
)

print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()

## main

In [9]:
lp.seed_everything(42) 

best_ckpt_paths, fold_metrics = [], []

####################################
train_days, val_days = folds_by_day[fold_id]
####################################

print(f"[fold {fold_id}] train {train_days[0]}..{train_days[-1]} ({len(train_days)} days), "
    f"val {val_days[0]}..{val_days[-1]} ({len(val_days)} days)")

# 明确日期：
train_start_date = int(train_days[0])
train_end_date   = int(train_days[-1])
val_start_date   = int(val_days[0])
val_end_date     = int(val_days[-1])      

# 将数据转为 Pandas 以供 TFT 使用
df = lf_pca.collect().to_pandas()


df[G_SYM] = df[G_SYM].astype(str).astype("category")
df.sort_values([G_SYM, "time_idx"], inplace=True)

Seed set to 42


[fold 0] train 1400..1534 (135 days), val 1542..1569 (28 days)


In [ ]:
if False:
    # STATIC 与 KNOWN 不重叠
    assert not (set(STATIC_REALS) & set(KNOWN_REALS))
    # 这些列都在数据里
    need = set(KNOWN_REALS) | set(STATIC_REALS) | set(KNOWN_CATEGORIES) | {G_SYM, "time_idx", TARGET_COL}
    missing = [c for c in need if c not in pdf_data.columns]
    assert not missing, f"missing columns: {missing}"


In [10]:
# 明确 indexes:
train_end_idx = df.loc[df[G_DATE] == train_end_date, "time_idx"].max()
val_start_idx = df.loc[df[G_DATE] == val_start_date, "time_idx"].min()
val_end_idx   = df.loc[df[G_DATE] == val_end_date, "time_idx"].max()
assert pd.notna(train_end_idx) and pd.notna(val_start_idx) and pd.notna(val_end_idx), "train/val idx not found"
train_end_idx, val_start_idx, val_end_idx = int(train_end_idx), int(val_start_idx), int(val_end_idx)
print(f"[fold {fold_id}] train idx up to {train_end_idx}, val idx {val_start_idx}..{val_end_idx}")  


[fold 0] train idx up to 517879, val idx 524656..551759


In [11]:
if False: 
    pdf_data = pdf_data[TRAIN_COLS]
    # 定义 identity scalers
    UNSCALE_COLS = KNOWN_REALS #+ STATIC_REALS
    identity_scalers = {name: None for name in UNSCALE_COLS}

meta_cols = [G_SYM, "time_idx", G_DATE, G_TIME, WEIGHT_COL, TARGET_COL]
pca_cols = [c for c in df.columns if c not in meta_cols]
identity_scalers = {name: None for name in pca_cols}

# 构建训练集 timeseries dataset
train_ds = TimeSeriesDataSet(
    df.loc[df["time_idx"] <= train_end_idx],
    time_idx="time_idx",
    target=TARGET_COL,
    group_ids=[G_SYM],
    weight=WEIGHT_COL,
    
    max_encoder_length=ENC_LEN, min_encoder_length=ENC_LEN,
    max_prediction_length=PRED_LEN, min_prediction_length=PRED_LEN,
    
    time_varying_known_reals=pca_cols,
    
    static_categoricals=[G_SYM],
    categorical_encoders={G_SYM: NaNLabelEncoder(add_nan=True)},
    
    allow_missing_timesteps=True,
    
    target_normalizer=GroupNormalizer(
        method="standard", groups=[G_SYM]),
    scalers=identity_scalers,
    
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

# 验证集复用 train_ds 的所有 encoders/normalizer（不泄漏）

val_ds = TimeSeriesDataSet.from_dataset(
    train_ds,
    df.loc[df["time_idx"].between(val_start_idx-ENC_LEN, val_end_idx, inclusive="both")],
    min_prediction_idx=val_start_idx,
    stop_randomization=True,
    predict=False
)

#del pdf_data, lf_clean, lf_with_idx; gc.collect()

In [ ]:
if False:
    
    lp.seed_everything(42)
    # 配置：每个 epoch 想训练多少个“样本/窗口”
    # 任选其一：按比例或固定数量
    SAMPLE_FRACTION = 0.1      # 例：每个 epoch 只随机抽取 20% 的窗口
    # SAMPLES_PER_EPOCH = 10000  # 或者：固定每个 epoch 抽 1e4 个窗口

    # 计算 num_samples（即本 epoch 的样本数）
    _train_total = len(train_ds); _val_total = len(val_ds)
    num_samples = max(1, int(_train_total * SAMPLE_FRACTION))
    # 如果用固定数量就改为：
    # num_samples = min(_train_total, SAMPLES_PER_EPOCH)

    # 打印epoch 样本数
    print(f"train full samples: {_train_total}, val full samples: {_val_total}")

    # 关键：带放回的随机采样器（每个 epoch 都会重新抽样）
    train_sampler = RandomSampler(train_ds, replacement=True, num_samples=num_samples)


    train_loader = train_ds.to_dataloader(train=True, batch_size=BATCH_SIZE, num_workers=12, pin_memory=True, persistent_workers=True, prefetch_factor=2, shuffle=False, sampler=train_sampler, drop_last=True)

    val_loader   = val_ds.to_dataloader(train=False, batch_size=BATCH_SIZE*8, num_workers=12, pin_memory=True, persistent_workers=True, prefetch_factor=2, shuffle=False)

    n_train_batches = len(train_loader)
    print(f"train_loader batches = {n_train_batches}")
    n_val_batches = len(val_loader)
    print(f"val_loader batches = {n_val_batches}")

    print(f"[{_now()}] data loaders ready")

In [12]:
train_loader = train_ds.to_dataloader(train=True, batch_size=BATCH_SIZE, num_workers=12, pin_memory=True, persistent_workers=True, prefetch_factor=2, shuffle=True, drop_last=True)

val_loader   = val_ds.to_dataloader(train=False, batch_size=BATCH_SIZE*8, num_workers=12, pin_memory=True, persistent_workers=True, prefetch_factor=2, shuffle=False, drop_last=True) # 先用 drop_last

n_train_batches = len(train_loader)
print(f"train_loader batches = {n_train_batches}")
n_val_batches = len(val_loader)
print(f"val_loader batches = {n_val_batches}")

print(f"[{_now()}] data loaders ready")

train_loader batches = 76226
val_loader batches = 2011
[2025-10-27 12:09:57] data loaders ready


In [ ]:
del df; gc.collect()

## 优化学习率

In [ ]:
from lightning.pytorch.tuner import Tuner

pl.set_random_seed(42)

# 1) 运行前清理缓存（可选）
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

# 2) 给 LR finder 专用的小 batch loader
#    不要动正式的 train_loader/val_loader
# 3) 混合精度（优先 bf16，其次 fp16）
trainer_lr = lp.Trainer(
    accelerator="gpu",
    gradient_clip_val=0.1,
)

# 4) 可选：只为 LR finder 构建一个“轻量化”的 TFT（不影响正式模型）
tft_lr = TemporalFusionTransformer.from_dataset(
    train_ds,
    learning_rate=LR,
    hidden_size=HIDDEN,             # 降低一点隐藏维度
    attention_head_size=HEADS,       # 降低头数
    hidden_continuous_size=HIDDEN // 2,
    dropout=DROPOUT,
    loss=QuantileLoss(),
    optimizer="adamw",
)

# 5) 运行 LR finder，限制搜索步数
res = Tuner(trainer_lr).lr_find(
    tft_lr,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    min_lr=1e-6,
    max_lr=1e-2,         
)

In [ ]:
print(f"suggested learning rate: {res.suggestion()}")
fig = res.plot(show=True, suggest=True)
fig.show()


## 模型训练

In [13]:
from zoneinfo import ZoneInfo   # py>=3.9 自带
import types

LOCAL_TZ = ZoneInfo("Europe/Copenhagen")     # 换成你想用的时区，如 "Asia/Shanghai"
ts = datetime.now(LOCAL_TZ).strftime("%Y-%m-%d_%H-%M-%S")


tft = TemporalFusionTransformer.from_dataset(
    train_ds,
    learning_rate=LR, # 只是占位，实际由 OneCycle 控
    hidden_size=HIDDEN,
    attention_head_size=HEADS,
    dropout=DROPOUT,
    hidden_continuous_size=HIDDEN//2,
    loss=QuantileLoss(quantiles=[0.5]),                            # 点预测 OK，会自动用 weight                     
    optimizer="adamw",
)
# 打印模型参数
print(f"fold_id = {fold_id}")
print(f"train_lo = {train_lo}, train_hi = {train_hi}")
print(f"val_lo = {val_lo}, val_hi = {val_hi}")
print(f"target_col = {TARGET_COL}")
print(f"weight_col = {WEIGHT_COL}")
print(f"encode length = {ENC_LEN}, pred length = {PRED_LEN}")
print(f"fake lr = {LR}, batch size = {BATCH_SIZE}")
print(f"hiden size = {HIDDEN}, heads = {HEADS}, dropout = {DROPOUT}")
print(f"max epochs = {MAX_EPOCHS}")
print(f"gradient clip val = {GCV}")

steps_per_epoch = 1000  
def _cfg(self):
    opt = torch.optim.AdamW(self.parameters(), lr=LR, weight_decay=1e-2)
    sch = torch.optim.lr_scheduler.OneCycleLR(
        opt, max_lr=LR,
        epochs=MAX_EPOCHS, steps_per_epoch=steps_per_epoch,
        pct_start=0.3, anneal_strategy='cos',
        div_factor=10, final_div_factor=100,
        cycle_momentum=False,  # AdamW 建议关
    )
    return {
        "optimizer": opt,
        "lr_scheduler": {"scheduler": sch, "interval": "step"}  # 每 step 调度
    }

tft.configure_optimizers = types.MethodType(_cfg, tft)


ckpt_dir_fold = Path(CKPTS_DIR) / f"fold_{fold_id}"
ckpt_dir_fold.mkdir(parents=True, exist_ok=True)

RUN_NAME = f"pca_time_{ts}adamw_onecycle_maxlr{LR:g}_bs{BATCH_SIZE}_enc{ENC_LEN}_hidden{HIDDEN}_heads{HEADS}"
TEMP_LOG_DIR = Path("./tft_logs"); TEMP_LOG_DIR.mkdir(parents=True, exist_ok=True)
logger = TensorBoardLogger(save_dir=TEMP_LOG_DIR.as_posix(),version=RUN_NAME,default_hp_metric=False)

callbacks = [EarlyStopping(monitor="val_loss", mode="min", patience=5), 
            LearningRateMonitor()
            ] 

trainer = lp.Trainer(
    #fast_dev_run=1,
    max_epochs=MAX_EPOCHS,
    accelerator="gpu",
    devices=1,
    precision="16-mixed",
    enable_model_summary=True,
    limit_train_batches=1000,  # 用全部 batch 训练
    limit_val_batches=30,    # 用全部 batch 验证
    gradient_clip_val=GCV,          
    num_sanity_val_steps=0,
    callbacks=callbacks,   # ← 记得把 lr_logger 放进来
    logger=logger,
    accumulate_grad_batches=ACCUM,
)

Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


fold_id = 0
train_lo = 1400, train_hi = 1534
val_lo = 1542, val_hi = 1569
target_col = responder_6
weight_col = weight
encode length = 128, pred length = 1
fake lr = 0.0005, batch size = 64
hiden size = 64, heads = 2, dropout = 0.2
max epochs = 40
gradient clip val = 0.1


In [14]:
import torch

def patch_tft_mask_bias(model):
    hit = 0
    for n, m in model.named_modules():
        if hasattr(m, "mask_bias"):
            old = getattr(m, "mask_bias")
            if isinstance(old, (int, float)) and old < -1e8:
                setattr(m, "mask_bias", -65504.0)   # 关键：FP16 最小可表示
                hit += 1
    print(f"[TFT patch] mask_bias replaced in {hit} module(s).")
    # 诊断输出（确认真的改到了）
    for n, m in model.named_modules():
        if hasattr(m, "mask_bias"):
            print(" -", n, type(m.mask_bias), m.mask_bias)

patch_tft_mask_bias(tft)


[TFT patch] mask_bias replaced in 2 module(s).
 - multihead_attn <class 'float'> -65504.0
 - multihead_attn.attention <class 'float'> -65504.0


In [15]:
trainer.fit(tft, train_dataloaders=train_loader, val_dataloaders=val_loader)#####2

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params | Mode 
------------------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0      | train
1  | logging_metrics                    | ModuleList                      | 0      | train
2  | input_embeddings                   | MultiEmbedding                  | 520    | train
3  | prescalers                         | ModuleDict                      | 12.5 K | train
4  | static_variable_selection          | VariableSelectionNetwork        | 20.9 K | train
5  | encoder_variable_selection         | VariableSelectionNetwork        | 1.7 M  | train
6  | decoder_variable_selection         | VariableSelectionNetwork        | 1.7 M  | train
7  | static_context_variable_selection  | GatedResidualNetwork            | 16.8 K | train
8  | static_context_initial_hidden_lstm |

Epoch 14: 100%|██████████| 1000/1000 [31:49<00:00,  0.52it/s, v_num=ads2, train_loss_step=1.240, val_loss=1.550, train_loss_epoch=1.360]


In [ ]:
# http://localhost:16006

In [ ]:
# 8.6 callbacks/logger/trainer
ckpt_dir_fold = Path(CKPTS_DIR) / f"fold_{fold_id}"
ckpt_dir_fold.mkdir(parents=True, exist_ok=True)

callbacks = [EarlyStopping(monitor="val_loss", mode="min", patience=6),
            ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1, dirpath=ckpt_dir_fold.as_posix(), filename=f"fold{fold_id}-tft-best-{{epoch:02d}}-{{val_loss:.5f}}", save_on_train_epoch_end=False),
            ] # LearningRateMonitor(logging_interval="step"),
RUN_NAME = f"without_static_f{fold_id}_E{MAX_EPOCHS}_lr{LR:g}_bs{BATCH_SIZE}_enc{ENC_LEN}_dec{DEC_LEN}_{datetime.now():%Y%m%d-%H%M%S}"

TEMP_LOG_DIR = Path("./tft_logs")
TEMP_LOG_DIR.mkdir(parents=True, exist_ok=True)

logger = TensorBoardLogger(save_dir=TEMP_LOG_DIR.as_posix(),name="tft",version=RUN_NAME,default_hp_metric=False)

trainer = lp.Trainer(max_epochs=3,
                    accelerator="gpu",
                    devices=1,
                    precision="bf16-mixed",
                    enable_model_summary=True,
                    gradient_clip_val=1.0,
                    gradient_clip_algorithm="norm",
                    #fast_dev_run=1,
                    limit_train_batches=1.0,
                    limit_val_batches=1.0,
                    val_check_interval=0.5,
                    num_sanity_val_steps=0,
                    log_every_n_steps=200,
                    callbacks=callbacks,
                    logger=logger,
                    #accumulate_grad_batches=1,
                    )

tft = TemporalFusionTransformer.from_dataset(
    train_ds,
    learning_rate=LR,
    hidden_size=HIDDEN,
    attention_head_size=HEADS,
    dropout=DROPOUT,
    hidden_continuous_size=HIDDEN // 2,
    loss=RMSE(),
    logging_metrics=[],
    optimizer=torch.optim.AdamW,
    optimizer_params={"weight_decay": 1e-4},
    reduce_on_plateau_patience=3, 
)
trainer.fit(
    tft,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
    )

In [16]:
ckpt_cb = next(cb for cb in callbacks if isinstance(cb, ModelCheckpoint))
best_path = ckpt_cb.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_path)

predictions = best_tft.predict(
    val_loader,
    return_y=True,
    trainer_kwargs=dict(accelerator="gpu")
)
y_pred = predictions.output
y_true, w = predictions.y

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


: 

In [ ]:
num = (w * (y_true - y_pred).pow(2)).sum()
den = (w * y_true.pow(2)).sum()

wr2 = 1.0 - num / (den + eps)
print(f"wr2 after training: {wr2.item():.6f}")

In [ ]:
val_ds.get_parameters

In [ ]:
num = (torch.square(y_true - y_pred) * w).sum()
den = (torch.square(y_true) * w).sum()  
wr2 = 1 - num / den
print(f"wr2 after training: {wr2}")